## Sample Data

In [1]:
import json

In [2]:
def read_json(file):
    with open(file) as f:
        data = json.load(f)
    return data

In [3]:
rule_based_title_comment_data = read_json("../sample_data/rule_based_title_comment_data.json")
sample_summarized_pnl_commentaries = read_json("../sample_data/sample_summarized_pnl_commentaries.json")
sample_executive_summary = read_json("../sample_data/sample_executive_summary.json")

In [ ]:
# # Save the sample_summarized_pnl_commentaries to a file
# with open("../sample_data/sample_summarized_pnl_commentaries.json", "w") as f:
#     json.dump(sample_summarized_pnl_commentaries, f, indent=4)

In [4]:
# Remove COMMENT from rule_based_title_comment_data
for key in rule_based_title_comment_data:
    del key['COMMENT']

In [5]:
title_comment_template = "For Buisness <BUIS>, on  <DATE>, driven by <NET>$  <FACTOR> <PROF_LOSS> to PL on <CUR> Currency on Portfolio <PF> and Desk <DSK>"

## Pompting And Gemeini 

In [6]:
import os
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from utils.gemini_service import GeminiModel, GeminiJsonEngine, GeminiSimpleChatEngine
import time

/Users/debasmitroy/Desktop/programming/gemini-agent-assist/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/debasmitroy/Desktop/programming/gemini-agent-assist/key.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "hackathon0-project"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

### Refining the sample summaries

In [8]:
gemini_simple_chat_engine = GeminiSimpleChatEngine(model_name="gemini-1.5-flash-002", 
                                                   temperature=0.5, 
                                                   max_output_tokens=512,
                                                   systemInstructions="You are an expert financial bot. You will be given a financial report and you need to refine the report. Keep everything in a single large paragraph. Dont use any markdown or bullet points. ")

Initialized GeminiModel with model gemini-1.5-flash-002 , project hackathon0-project, location us-central1


In [ ]:
refined_sample_summarized_pnl_commentaries = []
for sample_comment in sample_summarized_pnl_commentaries:
    _refinement_prompt = [
        f"Given financial report: {sample_comment}",
        f"Please refine the financial report in a more readable and meangingful way without losing any important information and entitites and technical/financial terms. Dont unnecessarily change the meaning of the report and dont increase the length of the report. "
    ]
    refined_sample_comment = gemini_simple_chat_engine(_refinement_prompt)
    refined_sample_summarized_pnl_commentaries.append(refined_sample_comment)
    time.sleep(5)

In [10]:
len(refined_sample_summarized_pnl_commentaries), refined_sample_summarized_pnl_commentaries[0]

(2,
 "Derivatives trading showed a €2 million profit, a significant 92% decrease year-over-year (PY), primarily due to increased losses in the book and held (B&H) portfolio.  Individual strategy performance included G10 currencies (€3 million profit), Metals (€2 million profit), Exotics (€1 million loss), and Management (€2 million loss). New deals generated €16 million, driven by strong performance in G10 currencies (€7 million), Exotics (€4 million), Precious Metals (€3 million), Latin America (€2 million), and Asia (€1 million), although this represents a 30% decrease versus PY due to reduced trading activity during the holiday season.  Key market influences included lower-than-expected Consumer Price Index (CPI) data,  anticipation of future interest rate cuts, and the Bank of Japan's (BOJ) continued dovish monetary policy. The request for quote (RFQ) conversion rate improved to 10.7% from 8.7% PY, despite a 26% decline in client requests (26,000 vs 35,000 PY) and a 7% reduction in

### Query Genreation

In [11]:
class Query(BaseModel):
    """
    Represents a financial query involving placeholders for key financial attributes: 
    <BUIS>, <DATE>, <NET>, <FACTOR>, <PROF_LOSS>, <CUR>, <PF>, and <DSK>.  

    Example Queries:  
    - What are the <FACTOR>s that contributed the highest <NET> profit/loss?  
    - Which <CUR> currencies are driving the top-performing portfolios <PF>?  
    """
    query: str = Field(..., title="Financial query using placeholders <BUIS>, <DATE>, <NET>, <FACTOR>, <PROF_LOSS>, <CUR>, <PF>, and <DSK>.")

class FinancialQueries(BaseModel):
    """
    A collection of structured queries designed to generate financial summaries.  
    Each query should use placeholders (<FIELD>) instead of actual values.  
    """
    queries: List[Query] = Field(..., title="List of financial queries using placeholders <FIELD> instead of actual values.")

In [12]:
user_prompt_list = [
    "You are a financial assistant. Your task is to generate structured queries from given templates to create financial summaries.",
    
    f"Here is an example pattern for financial summaries: {refined_sample_summarized_pnl_commentaries[0]}.",
    
    f"You are provided with a list of rule-based templates in the format List[{title_comment_template}]. Extract meaningful queries from these templates.",
    
    """Generate at least 15 diverse queries that can be used to generate sample financial summaries.
    
    - The queries should focus on aggregations such as min, max, mean, and sum, or retrieve the top 5 / bottom 5 entities.  
    - Avoid queries that fetch all rows or list all entities without aggregation.  
    - Do not create separate queries for different aggregations on the same entity; instead, combine them into a single query.  
    - Dont ask for a particular value; instead, ask for a top k or bottom k value. Say, top 5 Business Units or bottom 5 Desks.
    - The queries should be sufficient to address the financial summary patterns mentioned above.  
    - Replace all field values with placeholders using the format <FIELD>. Do not include actual values.  
    - Do not summarize the data; just generate structured queries.""",
    
    "You must use the tool `FinancialQueries`. Your response must strictly follow the argument structure of `FinancialQueries`."
]


In [13]:
fin_qry_engine = GeminiJsonEngine(
    model_name="gemini-2.0-flash-001",
    basemodel=FinancialQueries,
    temperature=0.8,
    max_output_tokens=500,
    systemInstructions=None
)

Initialized GeminiModel with model gemini-2.0-flash-001 , project hackathon0-project, location us-central1


In [14]:
fin_quries = fin_qry_engine(
    user_query=user_prompt_list
)

In [15]:
fin_quries[0]['queries']

[{'query': 'What are the top 5 <FACTOR>s that contributed the highest <NET> profit/loss?'},
 {'query': 'Which <CUR> currencies are driving the top-performing portfolios <PF>?'},
 {'query': 'What are the bottom 5 performing <BUIS> business units based on <NET> profit/loss?'},
 {'query': 'List the top 5 <DSK> desks with the largest <PROF_LOSS> losses.'},
 {'query': 'What is the average <NET> profit/loss for each <BUIS> business unit on <DATE>?'},
 {'query': 'Which <PF> portfolios had the highest <NET> profit in <CUR>?'},
 {'query': 'What are the top 5 <FACTOR>s impacting <NET> profit/loss across all <BUIS> business units?'},
 {'query': 'What is the overall <NET> profit/loss trend for <BUIS> business unit <BUIS> over the past <DATE>?'},
 {'query': 'Which <CUR> currencies have the lowest average <NET> profit/loss across all <PF> portfolios?'},
 {'query': 'What is the distribution of <NET> profit/loss across different <DSK> desks?'},
 {'query': 'What are the top 5 <BUIS> business units with

### Code Generation Based On Query

In [45]:
from sqlalchemy import create_engine, Column, String, Table, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import pandas as pd
import re

class InMemoryDB:
    def __init__(self):
        self.engine = create_engine('sqlite:///:memory:', echo=False)
        self.metadata = MetaData()
        self.Session = sessionmaker(bind=self.engine)
        self.session = self.Session()

    def create_table(self, table_name, columns):
        """Creates a table dynamically based on given column names."""
        table = Table(
            table_name, self.metadata,
            *[Column(col, String) for col in columns],  # Corrected column definition
        )
        table.create(self.engine)

    def insert_data(self, table_name, data):
        """Inserts a row into the table using parameterized queries."""
        placeholders = ", ".join([f":{key}" for key in data.keys()])
        query = text(f"INSERT INTO {table_name} ({', '.join(data.keys())}) VALUES ({placeholders})")
        self.session.execute(query, data)
        self.session.commit()

    def query_data(self, query):
        """Executes a SELECT query and returns results with column names."""
        result = self.session.execute(text(query))
        columns = result.keys()  # Get column names
        data = result.fetchall()  # Get data rows
        return list(columns), data  # Return both columns and data

    def __del__(self):
        self.session.close()

In [46]:
# Initialize DB and create table
inmemory_db = InMemoryDB()
inmemory_db.create_table("title_data", rule_based_title_comment_data[0].keys())

# Insert data
for data in rule_based_title_comment_data:
    inmemory_db.insert_data("title_data", data)

# Query and print results
cols, results = inmemory_db.query_data("SELECT * FROM title_data")

cols, results[:2]

(['BUIS', 'DATE', 'NET', 'FACTOR', 'PROF_LOSS', 'CUR', 'PF', 'DSK'],
 [('CEEMAEA', '2023-06-01', '1359240', 'BondBasis', 'LOSS', 'INR', 'American London CEEMAEA Portfolio', 'LATAM/NYC DSK'),
  ('CEEMAEA', '2023-06-01', '1697695', 'FXDelta', 'PROFFIT', 'GBP', 'European CEEMAEA Portfolio', 'US/LDN DSK')])

In [18]:
# Print the schema
print(inmemory_db.metadata.tables)

FacadeDict({'title_data': Table('title_data', MetaData(), Column('BUIS', String(), table=<title_data>), Column('DATE', String(), table=<title_data>), Column('NET', String(), table=<title_data>), Column('FACTOR', String(), table=<title_data>), Column('PROF_LOSS', String(), table=<title_data>), Column('CUR', String(), table=<title_data>), Column('PF', String(), table=<title_data>), Column('DSK', String(), table=<title_data>), schema=None)})


In [19]:
class SQLScript(BaseModel):
    """
    SQL Script to query data from the given table. You have to use this tool to generate the SQL script.
    """
    sql_script: str = Field(..., title="SQL Script to query data from the given table.")
    columns: List[str] = Field(..., title="Which columns are being projected in the SQL script.")
    description: str = Field(..., title="What does the SQL script do in Finance Analyst's perspective")

In [21]:
sql_query_engine = GeminiJsonEngine(
    model_name="gemini-2.0-flash-001",
    basemodel=SQLScript,
    temperature=0.8,
    max_output_tokens=500,
    systemInstructions=None
)



Initialized GeminiModel with model gemini-2.0-flash-001 , project hackathon0-project, location us-central1


In [50]:
pd.DataFrame(rule_based_title_comment_data).head()

,BUIS,DATE,NET,FACTOR,PROF_LOSS,CUR,PF,DSK
0,CEEMAEA,2023-06-01,1359240,BondBasis,LOSS,INR,American London CEEMAEA Portfolio,LATAM/NYC DSK
1,CEEMAEA,2023-06-01,1697695,FXDelta,PROFFIT,GBP,European CEEMAEA Portfolio,US/LDN DSK
2,LATAM,2023-06-01,125891,BondBasis,LOSS,EUR,LATAM Emerging Portfolio,LATAM/NYC DSK
3,CEEMAEA,2023-06-02,80205,BondBasis,LOSS,EUR,American London CEEMAEA Portfolio,US/LDN DSK
4,LATAM,2023-06-02,1287150,IRDelta,LOSS,GBP,American London CEEMAEA Portfolio,US/LDN DSK


In [41]:
qry_index = -2

print(f"Use case: {fin_quries[0]['queries'][qry_index]['query']}")

user_sql_prompt = [
    f"You are a SQL expert. Your task is to write a SQL script to query data from the given table. Note: you are generating a SQL script for SQLLite's python library. You must be careful while writing complex queries as it is very sensitive.",
    f"Library specific notes: STDDEV is not supported in SQLLite. You can use AVG and SUM to calculate the standard deviation.",
    f"Here is the schema of the table `title_data`: {inmemory_db.metadata.tables}",
    f"Here is the are the first few rows of the table `title_data`: {pd.DataFrame(rule_based_title_comment_data).head()}",
    f"User is trying to answer the following query: {fin_quries[0]['queries'][qry_index]['query']}",
    f"Write a SQL script to answer the query using the tool `SQLScript`. Your answer must follow the argument strucure of the tool `SQLScript`. You are encouraged to use compound and complex SQL queries to answer the query."
]


sql_qry_result = sql_query_engine(
    user_query=user_sql_prompt
)

# sql_script = re.search(r'```sql(.*?)```', sql_qry_result[0]['sql_script'], re.DOTALL).group(1).strip()
sql_script = sql_qry_result[0]['sql_script']

print(f"SQL script: {sql_script}")

sql_result = inmemory_db.query_data(sql_script)

sql_result

Use case: Identify the top 5 <PF> portfolios with the greatest volatility in <NET> profit/loss.
SQL script: SELECT PF, SUM(NET * NET) - SUM(NET) * SUM(NET) / COUNT(NET) AS Volatility FROM title_data GROUP BY PF ORDER BY Volatility DESC LIMIT 5


[('European CEEMAEA Portfolio', 133105120587114),
 ('American London CEEMAEA Portfolio', 128560048600709),
 ('LATAM Emerging Portfolio', 111238796029134)]

NameError: name 'dasd' is not defined